In [ ]:
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import yaml

import MopsPlotter
import MopsReader
#import MopsAnalysis
from MopsTracker import MopsTracker
from MopsParameters import MopsParameters

% matplotlib inline

In [ ]:
dbname = 'testData/testsources.db'

In [ ]:
con = sql.connect(dbname)

In [ ]:
full = pd.read_sql_query("""
SELECT * FROM testsources
""", con)

In [ ]:
full

In [ ]:
fig, ax = plt.subplots(1,1, dpi=400)
ax.scatter(np.array(full.ra), np.array(full.dec));
fig.savefig("full.jpg")

In [ ]:
ssmids = pd.read_sql_query("""
SELECT DISTINCT ssmid FROM testsources
""", con)

In [ ]:
len(ssmids['ssmid'])

In [ ]:
num_ssm = random.sample(ssmids['ssmid'], 1)
sample = ""
for i in num_ssm:
    sample += str(i) + ', '
    
sample = '(' + sample[0:-2] + ')'

In [ ]:
sample = ("(3)")

In [ ]:
objs = pd.read_sql_query("""
SELECT * FROM testsources
WHERE ssmid IN %s
""" % (sample), con)

In [ ]:
objs

In [ ]:
objs.to_csv("moresample.txt", sep=" ", header=False, index=False)

In [ ]:
MopsPlotter.plotData(objs)

In [ ]:
! rm -rf nightly/
! rm -rf obshist/
! mkdir nightly
! mkdir obshist

In [ ]:
! python $MOPS_DIR/bin/splitByNight.py moresample.txt nightly obshist 

In [ ]:
! rm -rf run/

In [ ]:
! python ../runMops.py nightly run -w 15 #-cfg *.cfg

In [ ]:
parameters = yaml.load(file('/Users/joachim/repos/neosim/unittest/run/parameters.yaml','r'))
tracker = yaml.load(file('/Users/joachim/repos/neosim/unittest/run/tracker.yaml','r'))

In [ ]:
MopsPlotter.plotTracklets(objs, tracker.tracklets)

In [ ]:
MopsPlotter.plotTracklets(objs, tracker.collapsedTrackletsById)

In [ ]:
MopsPlotter.plotTracklets(objs, tracker.purifiedTrackletsById)

In [ ]:
MopsPlotter.plotTracklets(objs, tracker.finalTrackletsById)

In [ ]:
MopsPlotter.plotTracks(objs, tracker.tracks)

In [ ]:
import MopsAnalysis

In [ ]:
ssmids = MopsAnalysis.findSSMIDs(objs, [2,4,170])

In [ ]:
ssmids

In [ ]:
MopsAnalysis.checkSSMIDs(MopsAnalysis.findSSMIDs(objs, [2,4,170]))

In [ ]:
MopsAnalysis.countSSMIDs(objs)

In [ ]:
test = tracker.dets[2]

In [ ]:
df = pd.read_csv(test, sep=' ', header=None, names=['diaid', 'obshistid', 'ssmid', 'ra', 'dec', 'mjd', 'mag', 'snr'])

In [ ]:
import MopsReader

In [ ]:
df = MopsReader.readDetections(test)

In [ ]:
df

In [ ]:
from MopsObjects import diasource
from MopsObjects import tracklet
from MopsObjects import track
from MopsAnalysis import runAnalysis
import os
import time

In [ ]:
def checkSSMIDs(ssmids):
    uniqueIds = np.unique(ssmids)
    if len(uniqueIds) == 1:
        return True
    else:
        return False

def countSSMIDs(dataframe):
    return dataframe['ssmid'].nunique()


In [ ]:
def analyzeTracks(trackFile, detFile, idsFile, verbose=True):
    startTime = time.ctime()
    print "Starting analysis for %s at %s" % (os.path.basename(trackFile), startTime)
    
    # Create outfile to store results
    outFile = trackFile + ".results"
    outFileOut = open(outFile, "w")
    print "Writing results to %s" % (outFile)
    
    # Read detections into a dataframe
    dets_df = MopsReader.readDetections(detFile)
    
    trackFileIn = open(trackFile, "r")
    tracks = []
    diasource_dict = {}
    
    # Initalize success (or failure) counters
    total_tracks = 0
    true_tracks = 0
    false_tracks = 0
    unique_ssmids = countSSMIDs(dets_df)
    found_ssmids = {}
    
    # Examine each line in trackFile and read in every line
    #  as a track object. If track contains new detections (diasource)
    #  then add new source to diasource_dict. 
    for line in trackFileIn:
        # Found a track!
        total_tracks += 1
        new_track_diaids = MopsReader.readTrack(line)
        new_track = []
        
        # Look up each diaid in the track, check if diasource object exists.
        #  If it does exist then add it to the new track object, if not then create the object
        #  and update the diasource object dictionary.
        for diaid in new_track_diaids:
            ssmids = []
            if diaid in diasource_dict:
                ssmids.append(diasource_dict[diaid].ssmid)
                new_track.append(diasource_dict[diaid])
                
                if diasource_dict[diaid].ssmid in found_ssmids:
                    found_ssmids[diasource_dict[diaid].ssmid] += 1
                else:
                    found_ssmids[diasource_dict[diaid].ssmid] = 1
            
            else:
                new_diasource = dets_df.loc[diaid]
                new_diasource_obj = diasource(int(diaid), new_diasource['ssmid'],
                             new_diasource['obshistid'], new_diasource['ra'],
                             new_diasource['dec'], new_diasource['mjd'],
                             new_diasource['mag'], new_diasource['snr'])
                diasource_dict[diaid] = new_diasource_obj
                
                ssmids.append(diasource_dict[diaid].ssmid)
                new_track.append(new_diasource_obj)
                
        isTrue = checkSSMIDs(ssmids)  
        if isTrue:
            # Track is true! 
            true_tracks += 1
        else:
            # Track is false. 
            false_tracks += 1
            
        final_track = track(new_track) 
        final_track.isTrue = isTrue
        final_track.rms, final_track.raRes, final_track.decRes, final_track.distances = calcRMS(final_track.diasources)
        tracks.append(final_track)
        
    endTime = time.ctime()
    print "Finished analysis for %s at %s" % (os.path.basename(trackFile), endTime)

    outFileOut.write("Start time: %s\n" % (startTime))
    outFileOut.write("True tracks: %s\n" % (true_tracks))
    outFileOut.write("False tracks: %s\n" % (false_tracks))
    outFileOut.write("Total tracks: %s\n" % (total_tracks))
    outFileOut.write("Findable objects: %s\n" % (unique_ssmids))
    outFileOut.write("Found objects: %s\n" % (len(found_ssmids)))
    outFileOut.write("End time: %s\n" % (endTime))

    return true_tracks, false_tracks, total_tracks, unique_ssmids, found_ssmids, tracks

In [ ]:
t,tt,ttt,a,b,ts = analyzeTracks(tracker.tracks[0], tracker.dets[0], tracker.ids[0])

In [ ]:
ts[3].isTrue

In [ ]:
diasource_dict[int(diaid)] = diasource(int(diaid), int(new_diasource['ssmid']),
                             int(new_diasource['obshistid']), float(new_diasource['ra']),
                             float(new_diasource['dec']), float(new_diasource['mjd']),
                             float(new_diasource['mag']), float(new_diasource['snr']))

In [ ]:
while d[i].diaid != 4:
    p

In [ ]:
tracker.tracks[0]

In [ ]:
diasource?

In [ ]:
runAnalysis(parameters, tracker)

In [ ]:
def calcDegToRad(theta):
    return  theta*(np.pi/180.0)

def calcRadToDeg(theta):
    return theta*(180.0/np.pi)

def calcAngularDistance(a, b):
    """ return distance between a and b, where a and b are angles in degrees. """
    while abs(a - b) > 180:
        if a > b:
            b += 360.
        else:
            a += 360.
    return a - b

def convertToStandardDegrees(angle):
    while angle > 360.:
        angle -= 360.
    while angle < 0.:
        angle += 360.
    return angle

def calcGreatCircleDistance(ra0, dec0, ra1, dec1):
    """
    return the great-circle distance between two points on the sky,
    uses haversine formula
    """
    ra_dist = calcAngularDistance(ra0, ra1);
    dec_dist = calcAngularDistance(dec0, dec1);    
    # Convert all factors to radians
    ra_dist = calcDegToRad(convertToStandardDegrees(ra_dist));
    dec_dist = calcDegToRad(convertToStandardDegrees(dec_dist));
    dec0 = calcDegToRad(convertToStandardDegrees(dec0));
    dec1 = calcDegToRad(convertToStandardDegrees(dec1));
    r = 2*np.arcsin(np.sqrt((np.sin(dec_dist/2.))**2 + np.cos(dec0)*np.cos(dec1)*(np.sin(ra_dist/2))**2));
    # Back to degrees
    return calcRadToDeg(r);

def makeContiguous(angles):
    """ given a set of angles (say, RAs or Decs of observation) which
    span a fairly short arc but may actually cross the 0/360 line,
    make these contiguous by using negative angles or whatever is
    necessary.  if this set of angles does NOT span a short arc (>180
    deg) expect all hell to break loose."""
    a0 = angles[0]
    output = [a0]
    for angle in angles[1:]:
        while abs(angle - a0) > 180:
            if angle > a0:
                angle -= 360.
            else:
                angle += 360.
        output.append(angle)
    return output
    
def calcRMS(diasources):
    t0 = min(map(lambda x: x.mjd, diasources))
    ras = []
    decs = []
    mjds = []
    for diasource in diasources:
        ras.append(diasource.ra)
        decs.append(diasource.dec)
        mjds.append(diasource.mjd - t0)
    ras = makeContiguous(ras)
    decs = makeContiguous(decs)
    ras = np.array(ras)
    decs = np.array(decs)
    mjds = np.array(mjds)

    raFunc, raRes, rank, svd, rcond = np.polyfit(mjds, ras, 2, full=True)
    decFunc, decRes, rank, svd, rcond = np.polyfit(mjds, decs, 2, full=True)
    raFunc = np.poly1d(raFunc)
    decFunc = np.poly1d(decFunc)

    #now get the euclidean distance between predicted and observed for each point
    netSqDist = 0.0
    dists = []
    for i in range(len(mjds)):
        predRa = raFunc(mjds[i])
        predDec = decFunc(mjds[i])
        dist = calcGreatCircleDistance(predRa, predDec, ras[i], decs[i])
        dists.append(dist)
        if (dist > .1):
            print "Unexpected wierdness, diasource had angular distance of %f from best-fit curve prediction" % (dist)
            print "Predicted RA, Dec were ", predRa, predDec
            print "observed RA, Dec were ", ras[i], decs[i]
            print "all RAs were ", ras
            print "all decs were ", decs
        sqDist = dist**2
        #print "got euclidean distance was ", sqDist
        netSqDist += sqDist

    rms = np.sqrt(netSqDist / len(diasources))
    if (rms > .1):
        print "RMS error was %f " % (rms)
    return rms, raRes[0], decRes[0], dists

In [ ]:
ts[3].diasources

In [ ]:
a,b,c,d = calcRMS(ts[3].diasources)

In [ ]:
a

In [ ]:
len(d)

In [ ]:
test = runAnalysis(parameters, tracker)

In [ ]:
test.foundObjects

In [ ]:
temp.update({"1.0": temp[1.0] += 2})

In [1]:
import MopsReader

In [2]:
df = MopsReader.readDetections("/Users/joachim/repos/neosim/ldm156_0116_05/trackletsByNight/night_51040_through_51055.dets")

In [3]:
df

,obshistid,ssmid,ra,dec,mjd,mag,snr
diaid,,,,,,,
8517223,88677528,49231,234.712038,-36.373855,51039.965572,21.678118,8.404711
8517224,88677528,443381,236.046013,-36.865381,51039.965572,20.652854,21.608696
8517225,88677528,430544,234.885339,-37.522868,51039.965572,20.502174,24.825630
8517226,88677528,455586,236.394079,-35.924614,51039.965572,21.880187,6.977428
8517227,88677528,478231,236.330113,-36.769353,51039.965572,21.957860,6.495700
8517228,88677528,490450,236.110757,-34.010774,51039.965572,21.773899,7.695035
8517229,88677528,424429,235.824192,-34.419941,51039.965572,21.000429,15.689118
8517230,88677528,547976,235.569189,-34.622987,51039.965572,21.684067,8.358790
8517231,88677528,478185,237.655788,-35.852953,51039.965572,22.021924,6.123511
